# Bean Disease Classification - Google Colab Training

This notebook allows you to train the bean disease classification model on Google Colab's free GPU.

## Setup Instructions
1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator → GPU (T4)
2. **Run all cells**: Runtime → Run all
3. **Download models**: After training completes, download the models from the Files panel

## What This Does
- Clones the project repository
- Installs dependencies
- Trains a transfer learning model (Xception/EfficientNetV2/MobileNet)
- Generates Keras model + TFLite model for mobile deployment
- Provides downloadable model files

**Training Time**: ~15-30 minutes on T4 GPU (depends on configuration)

## 1. Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Clone the repository
!git clone https://github.com/martinr92/Bean-Disease-Classification.git
%cd Bean-Disease-Classification

In [ ]:
# Install dependencies
!pip install -q tensorflow==2.19.0
!pip install -q tensorflow-datasets==4.9.9
!pip install -q scikit-learn==1.6.1
!pip install -q numpy==2.1.3
!pip install -q Pillow==10.4.0

print("✅ Dependencies installed")

## 2. Training Configuration

Adjust these parameters to customize your training:

In [ ]:
# Training Configuration
TRAINING_CONFIG = {
    # Model selection (XCEPTION, EFFICIENT_NET_V2, MOBILE_NET)
    "base_model": "XCEPTION",
    
    # Training epochs
    "epochs_pretrain": 5,      # Initial training with frozen base
    "epochs_finetune": 10,     # Fine-tuning with unfrozen layers
    
    # Learning rates
    "initial_lr": 0.1,         # Learning rate for pretraining
    "finetune_lr": 0.01,       # Learning rate for fine-tuning
    
    # Data
    "batch_size": 16,
    "train_size": 1034,
    "val_size": 133,
    "test_size": 128,
    
    # Other
    "random_seed": 42,
    "patience": 10,            # Early stopping patience
}

print("Training Configuration:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

## 3. Setup Output Directories

In [ ]:
import os

# Create output directory for models
OUTPUT_DIR = "/content/models"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ Output directory created: {OUTPUT_DIR}")

## 4. Train Model

This will take approximately 15-30 minutes depending on your configuration.

**Note**: Uses the same training code as local and cloud options!

In [ ]:
import sys
import os
import datetime

# Add src directory to Python path
src_path = '/content/Bean-Disease-Classification/src'
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Verify the path exists
if not os.path.exists(src_path):
    raise FileNotFoundError(f"Source directory not found: {src_path}. Make sure you ran the clone cell first.")

# Import training modules
from config.training_config import TrainingConfig, BaseModel, Optimizer, DatasetSource
from training.trainer import train_model_core  # Core training without MLflow

# Create configuration
base_model_map = {
    "XCEPTION": BaseModel.XCEPTION,
    "EFFICIENT_NET_V2": BaseModel.EFFICIENT_NET_V2,
    "MOBILE_NET": BaseModel.MOBILE_NET,
}

config = TrainingConfig(
    base_model=base_model_map[TRAINING_CONFIG["base_model"]],
    optimizer=Optimizer.SGD,
    dataset_source=DatasetSource.TENSORFLOW,
    epochs_pretrain=TRAINING_CONFIG["epochs_pretrain"],
    epochs_finetune=TRAINING_CONFIG["epochs_finetune"],
    initial_lr=TRAINING_CONFIG["initial_lr"],
    finetune_lr=TRAINING_CONFIG["finetune_lr"],
    batch_size=TRAINING_CONFIG["batch_size"],
    train_size=TRAINING_CONFIG["train_size"],
    val_size=TRAINING_CONFIG["val_size"],
    test_size=TRAINING_CONFIG["test_size"],
    random_seed=TRAINING_CONFIG["random_seed"],
    patience=TRAINING_CONFIG["patience"],
    experiment_name="bean_disease_colab",
    run_name=f"colab_training_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}",
    mlflow_tracking_uri="",  # No MLflow on Colab
)

print("="*60)
print("Starting training...")
print(f"Model: {TRAINING_CONFIG['base_model']}")
print(f"Epochs: {TRAINING_CONFIG['epochs_pretrain']} + {TRAINING_CONFIG['epochs_finetune']}")
print("="*60)
print()

# Train model using same code as local/cloud
try:
    result = train_model_core(config, output_dir=OUTPUT_DIR)
    
    print("\n" + "="*60)
    print("✅ TRAINING COMPLETED SUCCESSFULLY!")
    print("="*60)
    print(f"Test Accuracy: {result['metrics']['test_accuracy']:.4f}")
    print(f"Test Loss:     {result['metrics']['test_loss']:.4f}")
    print(f"TFLite Size:   {result['tflite_size_mb']:.2f} MB")
    print(f"Models saved:  {OUTPUT_DIR}")
    print("="*60)
except Exception as e:
    print(f"\n❌ Training failed: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

## 5. Download Models

Run this cell to download both models to your computer.

In [ ]:
from google.colab import files
import os

# Download Keras model
keras_model_path = os.path.join(OUTPUT_DIR, "bean_disease_model.keras")
if os.path.exists(keras_model_path):
    print("Downloading Keras model...")
    files.download(keras_model_path)
    print("✅ Keras model downloaded")

# Download TFLite model
tflite_model_path = os.path.join(OUTPUT_DIR, "bean_disease_model.tflite")
if os.path.exists(tflite_model_path):
    print("Downloading TFLite model...")
    files.download(tflite_model_path)
    print("✅ TFLite model downloaded")

print("\n📦 All models downloaded! You can now use them in your applications.")

### Alternative: Download via Files Panel

You can also manually download the models:
1. Click the folder icon on the left sidebar
2. Navigate to `/content/models/`
3. Right-click on the model files and select "Download"

**Files:**
- `bean_disease_model.keras` - Full Keras model
- `bean_disease_model.tflite` - TensorFlow Lite model for mobile

## 6. Model Evaluation Results

View your model's performance:

In [ ]:
# Display training results
if 'result' in locals():
    print("="*60)
    print("TRAINING RESULTS")
    print("="*60)
    metrics = result['metrics']
    print(f"Final Training Accuracy:   {metrics['final_train_accuracy']:.4f}")
    print(f"Final Validation Accuracy: {metrics['final_val_accuracy']:.4f}")
    print(f"Test Accuracy:             {metrics['test_accuracy']:.4f}")
    print(f"Test Loss:                 {metrics['test_loss']:.4f}")
    print("="*60)
    print(f"TFLite Model Size:         {result['tflite_size_mb']:.2f} MB")
    print("="*60)
else:
    print("⚠️ Training not completed yet. Run the training cell first.")

## Next Steps

### Using Your Trained Model

**Keras Model (.keras):**
- Use for further training or experimentation
- Deploy to cloud services (TensorFlow Serving, Cloud Run)
- Use in Python applications

**TFLite Model (.tflite):**
- Deploy to Android/iOS mobile apps
- Use on edge devices (Raspberry Pi, etc.)
- Optimized for inference on resource-constrained devices

### Training Options Comparison

| Option | GPU | Cost | Setup | MLflow Tracking | Same Code |
|--------|-----|------|-------|-----------------|----------|
| **Local** | Your GPU | $0 | Complex (Docker, NVIDIA Toolkit) | ✅ Yes | ✅ Yes |
| **Colab** | Free T4 | $0 (free tier) | Simple (just upload notebook) | ❌ No | ✅ Yes |
| **Cloud** | GCP GPU | ~$1-2/hour | Medium (Cloud Run setup) | ✅ Yes | ✅ Yes |

### Repository
- GitHub: https://github.com/martinr92/Bean-Disease-Classification
- Full documentation in README.md

### Key Point
✅ **This notebook uses the exact same training code (`train_model_core`) as local and cloud options!**
- The only difference is MLflow tracking (enabled for local/cloud, disabled for Colab)
- This ensures consistent results across all training environments